# 1. Installation of requirements

In [1]:
# required packages and basic function(s)
from rdkit import Chem
from mordred import Calculator, descriptors
import pandas as pd
import numpy as np
import math
import random
import tqdm
import os
import shutil
import yaml
from pathlib import Path
from importlib import resources
import subprocess
import logging
from multiprocessing import freeze_support

logging.basicConfig(level=logging.INFO)
freeze_support()

def safe_div(value, factor, feature_name, printed_flag):
    try:
        return value / np.float64(factor)
    except Exception:
        # only print the first time this feature fails
        if not printed_flag[0]:
            print(f"{feature_name} unable to be calculated, setting invalid values to 0")
            printed_flag[0] = True
        return 0

# 2. Splitting data into test/train splits


The pre-trained AGILE deep learning model is provided in `AGILE/ckpt/pretrained_agile_60k` and will be fine-tuned on five cross-validation splits.

The data provided in [repo](https://github.com/bowang-lab/AGILE) was split (80% train/20% validation) randomly to create these splits.

In this section we will split the data.

In [4]:
# split AGILE data (Morgan fingerprints included)
random.seed(0)
no_splits = 5

# read from ../LNPDB/data/LNPDB_for_AGILE/data
df = pd.read_csv("../data/LNPDB_for_AGILE/AGILE/data/data/finetuning_set_smiles_plus_features.csv")

def create_df(name):
    name = pd.DataFrame()
    return name

output_dataframes=[]
for i in range(no_splits):
    output_dataframes.append(create_df(f"df{i}"))

complement_dataframes=[]
for k in range(no_splits):
    complement_dataframes.append(create_df(f"df{k}c"))

for index in range(len(df)):
    row = df.iloc[[index]]
    assignment = random.randint(0,no_splits-1)
    while len(output_dataframes[assignment])>=len(df)/no_splits:
        assignment = random.randint(0,no_splits-1)
    output_dataframes[assignment] = pd.concat([output_dataframes[assignment], row], ignore_index=True)
    for complement_assignment in range(no_splits):
        if complement_assignment != assignment:
            complement_dataframes[complement_assignment] = pd.concat([complement_dataframes[complement_assignment], row], ignore_index=True)

# save to ../LNPDB/data/LNPDB_for_AGILE/cv_splits
cv_dir = Path("../data/LNPDB_for_AGILE/cv_splits")
cv_dir.mkdir(parents=True, exist_ok=True)

for j, dataframe in enumerate(output_dataframes):
    dataframe.to_csv(cv_dir / f"df{j}_test.csv", index=False)

for k, dataframe in enumerate(complement_dataframes):
    dataframe.to_csv(cv_dir / f"df{k}_train.csv", index=False)

logging.info("Cross-validation splits saved to cv_splits/")


INFO:root:Cross-validation splits saved to cv_splits/


# 3. Finetuning models on cross-validation splits

The following section involves preparing files according to [AGILE](https://github.com/bowang-lab/AGILE).
    
Note that finetuning is nondeterministic especially with AGILE code being unseeded, and results may differ slightly between finetuned models.

The models and their results are now placed in respective folders in `LNPDB/data/LNPDB_for_AGILE/AGILE/finetune`.

## 3.1 Preparing finetune_LNPDB.py and YAML/split

In [5]:
lnpdb_agile_base = Path("../data/LNPDB_for_AGILE")
scripts_dir = Path("../data/LNPDB_for_AGILE/scripts")
agile_dir = Path("../data/LNPDB_for_AGILE/AGILE")

# Copy finetune script + yaml into AGILE, and patch YAML per split
finetune_py = scripts_dir / "finetune_LNPDB.py"
finetune_yaml_template = scripts_dir / "config_finetune.yaml"

# Ensure files exist
assert finetune_py.exists(), "finetune_LNPDB.py not found in scripts/"
assert finetune_yaml_template.exists(), "config_finetune.yaml not found in scripts/"

# Copy finetune_LNPDB.py into AGILE
shutil.copy(finetune_py, agile_dir / "finetune_LNPDB.py")

# Generate one YAML per split
finetune_config_dir = agile_dir / "finetune"
finetune_config_dir.mkdir(parents=True, exist_ok=True)

for i in range(5):
    yaml_target = finetune_config_dir / f"agile_lnp_hela_cv_{i}.yaml"
    with open(finetune_yaml_template, "r") as f:
        yaml_text = f.read()
    yaml_text = yaml_text.replace(
        "task_name: lnp_hela_with_feat",
        f"task_name: LNPDB_split_{i}"
    )
    with open(yaml_target, "w") as f:
        f.write(yaml_text)

logging.info("finetune_LNPDB.py and split YAMLs prepared in AGILE/")


INFO:root:finetune_LNPDB.py and split YAMLs prepared in AGILE/


## 3.2 Finetuning models on respective split train data

In [4]:
# finetune pre-trained AGILE model on 5 cross-validation splits
cv_number_list = [0,1,2,3,4]

finetune_dir = Path("finetune")
scripts_dir = Path("../scripts")
for i in cv_number_list:
    yaml_path = finetune_dir / f"agile_lnp_hela_cv_{i}.yaml"
    command = ["python", "finetune_LNPDB.py", str(yaml_path)]
    result = subprocess.run(command, cwd=agile_dir, capture_output=True, text=True)
    # uncomment to show the direct AGILE output and errors
    # print(result.stdout)
    # print(result.stderr)
    
logging.info("Finetuned models in AGILE/finetune named agile_lnp_hela_cv_{i}/")


INFO:root:Finetuned models in AGILE/finetune named agile_lnp_hela_cv_{i}/


# 4. Generating LNPDB data feature descriptors

To use the trained models to predict delivery efficacy for new LNP data, LNPDB data has been placed into the folder `LNPDB/data/LNPDB_for_AGILE/LNPDB_data`.

AGILE requires data to be processed into Mordred molecular feature descriptors, which are generated using [repo](https://github.com/mordred-descriptor/mordred) as described in the sixth cell of the notebook. Note that the "descriptors_full" output is from directly generating Mordred descriptors, while the "plus_features" output is with adjustment to required AGILE format. For all LNPDB data included in the paper, this repository already contains the corresponding Mordred descriptors in `LNPDB/data/LNPDB_for_AGILE/LNPDB_data`.

Our Mordred descriptors were validated by comparing generation from AGILE original SMILES and provided fingerprints. The procedure is to generate Mordred descriptors using solely the smiles and Experiment_value columns, and then compare the resulting descriptors with those provided by the AGILE [repo](https://github.com/bowang-lab/AGILE).

## 4.1 Generate descriptors

In [5]:
# Generate Mordred descriptors for heldout datasets and align with known features
def heldout_data(heldout_data_path: str,
                 heldout_data_output_folder: str,
                 known_data_path: str) -> None:
    """
    Generate Mordred descriptors for molecules in heldout_data, then select only
    those features that appear in known_data and append them to the heldout_data.
    Saves both the full descriptor table and the filtered heldout_data to CSV files.

    Parameters:
    - heldout_data_path: path to CSV with columns ['smiles', ...]
    - heldout_data_output_folder: folder to create for output files
    - known_data_path: path to CSV whose columns define the desired features
    """

    df_heldout = pd.read_csv(heldout_data_path)
    logging.info(f"Read heldout data with {len(df_heldout)} molecules.")
    try:
        mols = [Chem.MolFromSmiles(smi) for smi in df_heldout['smiles']]
    except:
        mols = [Chem.MolFromSmiles(smi) for smi in df_heldout['IL_SMILES']]
    calc = Calculator(descriptors, ignore_3D=True)
    df_desc = calc.pandas(mols, nproc=1)

    # clean columns
    if isinstance(df_desc.columns, pd.MultiIndex):
        df_desc.columns = [str(name) for name, _ in df_desc.columns]
    else:
        df_desc.columns = [str(c) for c in df_desc.columns]

    df_desc.replace([np.inf, -np.inf], pd.NA, inplace=True)

    if df_desc.isna().sum().sum() > 0:
        logging.warning(f"Detected {df_desc.isna().sum().sum()} NaN or overflow values after descriptor calculation.")

    os.makedirs(heldout_data_output_folder, exist_ok=True)
    dataset_name = Path(heldout_data_path).stem

    desc_full_path = os.path.join(heldout_data_output_folder,
                                  f"{dataset_name}_descriptors_full.csv")
    df_desc.to_csv(desc_full_path, index=False)
    logging.info(f"Full descriptor table saved to {desc_full_path}.")

    df_known = pd.read_csv(known_data_path)
    desired_cols = [col for col in df_known.columns if col.startswith('desc_')]
    logging.info(f"Found {len(desired_cols)} desired desc_* columns in known data.")

    # collect matching columns
    new_features = {}
    missing_features = []
    overflow_features = []
    # Extract feature names from desired columns
    scaling_factor = None
    for col in desired_cols:
        # some features used by AGILE are scaled
        if '/' in col:
            feature_name, scaling_factor = col[len('desc_'):].split('/')
        else:
            feature_name = col[len('desc_'):] 

        if feature_name in df_desc.columns:
            feature_values = df_desc[feature_name]
            printed_flag = [False]
            if feature_values.isna().any():
                logging.warning(f"Feature '{feature_name}' contains NaNs due to overflow during calculation.")
                overflow_features.append(feature_name)
            # else if feature_name == 'MAXssNH' or feature_name == 'MINssNH':
            elif scaling_factor:
                new_features[col] = [
                    safe_div(value, scaling_factor, feature_name, printed_flag)
                    for value in feature_values.values
                ]
            else:
                new_features[col] = [
                    safe_div(value, 1, feature_name, printed_flag)
                    for value in feature_values.values
                ]
            # uncomment to see all features during generation
            # logging.info(f"Added feature column '{col}' from descriptor '{feature_name}'.")
        else:
            # some features used by AGILE are log-Mordred features
            if feature_name == 'log_VR1_A' or feature_name == 'log_VR2_A' or feature_name == 'log_SdssC':
                feature_values = df_desc[f'{feature_name[4:]}']
                if feature_values.isna().any():
                    logging.warning(f"Feature '{feature_name}' contains NaNs due to overflow during calculation.")
                    overflow_features.append(f'log_{feature_name}')
                new_features[col] = [math.log10(value) if value>0 else 0 for value in feature_values.values]
                # uncomment to see all features during generation
                # logging.info(f"Added feature column '{col}' from descriptor '{feature_name}'.")
            else:
                logging.warning(f"Descriptor '{feature_name}' not found in calculated descriptors. Filling NaNs.")
                new_features[col] = [pd.NA] * len(df_heldout)
                missing_features.append(feature_name)

        scaling_factor = None

    df_new_features = pd.DataFrame(new_features)
    df_new_features = df_new_features.reindex(columns=desired_cols)
    df_heldout = pd.concat([df_heldout.reset_index(drop=True),
                            df_new_features.reset_index(drop=True)], axis=1)
    # df_heldout.rename(columns={'IL_SMILES': 'smiles'}, inplace=True)

    # 🔹 Ensure only known features are kept (drop extra Mordred descriptors)
    keep_cols = [c for c in df_heldout.columns if not c.startswith("desc_")] + desired_cols
    df_heldout = df_heldout[keep_cols]

    # concatenate smiles + labels (non-desc_) with filtered descriptors
    non_desc_cols = [c for c in df_heldout.columns if not c.startswith("desc_")]
    df_heldout_final = pd.concat(
        [df_heldout[non_desc_cols].reset_index(drop=True),
         df_new_features.reset_index(drop=True)],
        axis=1
    )

    # Ensure final output columns match the order in known_data
    ref_cols = list(df_known.columns)  # full reference column order
    non_desc_cols = [c for c in df_heldout_final.columns if not c.startswith("desc_")]

    # keep non-desc columns first, then descriptors in reference order
    ordered_cols = non_desc_cols + [c for c in ref_cols if c.startswith("desc_")]

    df_heldout_final = df_heldout_final.reindex(columns=ordered_cols)
    if 'IL_SMILES' in df_heldout_final.columns:
        df_heldout_final = df_heldout_final.rename(columns={'IL_SMILES': 'smiles'})
    
    heldout_out_path = os.path.join(heldout_data_output_folder, f"{dataset_name}_plus_features.csv")

    df_heldout_final.to_csv(heldout_out_path, index=False)

    logging.info(f"Augmented heldout data (with {len(desired_cols)} descriptors) saved to {heldout_out_path}")

    # Warn about missing features if any
    if missing_features:
        logging.warning(f"{len(missing_features)} features were missing: {missing_features}")

    if overflow_features:
        logging.warning(f"{len(overflow_features)} features contain NaN values due to overflow: {overflow_features}")

# Example call (edit datasets as needed)
heldout_data(
    "../data/LNPDB_for_AGILE/AGILE/data/data/finetuning_set_smiles_plus_features.csv", # only change this argument for each dataset
    #"../data/LNPDB_for_AGILE/LNPDB_data/BL_2023_heldout_data.csv",
    #"../data/LNPDB_for_AGILE/LNPDB_data/LM_2019_heldout_data.csv",
    #"../data/LNPDB_for_AGILE/LNPDB_data/SL_2020_heldout_data.csv",
    #"../data/LNPDB_for_AGILE/LNPDB_data/ZC_2023_heldout_data.csv",
    "../data/LNPDB_for_AGILE/outputs",
    "../data/LNPDB_for_AGILE/AGILE/data/data/finetuning_set_smiles_plus_features.csv"
)

INFO:root:Read heldout data with 1200 molecules.
/home/andersonxps/anaconda3/envs/agile_finaltest_lnpdb/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|███████████████████████████████████████| 1200/1200 [03:28<00:00,  5.75it/s]
INFO:root:Full descriptor table saved to ../data/LNPDB_for_AGILE/outputs/finetuning_set_smiles_plus_features_descriptors_full.csv.
INFO:root:Found 813 desired desc_* columns in known data.
INFO:root:Augmented heldout data (with 813 descriptors) saved to ../data/LNPDB_for_AGILE/outputs/finetuning_set_smiles_plus_features_plus_features.csv


## 4.2 Validate descriptor generation

In [6]:
# Compare generated Mordred descriptors vs. known features, with summary stats
def compare_descriptors(generated_path: str, known_path: str, n_check: int = 5,
                        atol: float = 1e-6, rtol: float = 1e-4):
    """
    Compare a generated Mordred descriptor CSV to a known training feature CSV.

    Parameters:
    - generated_path: path to heldout_data_output.csv (with descriptors)
    - known_path: path to finetuning_set_smiles_plus_features.csv
    - n_check: number of molecules to sample for detailed row comparison
    - atol, rtol: tolerances for numerical equality (absolute, relative)
    """
    df_gen = pd.read_csv(generated_path)
    df_known = pd.read_csv(known_path)

    # Select only descriptor columns (start with desc_)
    gen_cols = [c for c in df_gen.columns if c.startswith("desc_")]
    known_cols = [c for c in df_known.columns if c.startswith("desc_")]

    # Check overlap
    overlap = sorted(set(gen_cols) & set(known_cols))
    missing_in_gen = set(known_cols) - set(gen_cols)
    extra_in_gen = set(gen_cols) - set(known_cols)

    print(f"Total known features: {len(known_cols)}")
    print(f"Total generated features: {len(gen_cols)}")
    print(f"Overlap: {len(overlap)}")
    if missing_in_gen:
        print(f"Missing in generated: {sorted(list(missing_in_gen))[:10]} ...")
    if extra_in_gen:
        print(f"Extra in generated: {sorted(list(extra_in_gen))[:10]} ...")

    # Early exit if no overlap
    if not overlap:
        print("No overlapping descriptors to compare.")
        return

    # Align dataframes to overlap
    df_gen_overlap = df_gen[overlap].reset_index(drop=True)
    df_known_overlap = df_known[overlap].reset_index(drop=True)

    # Check shape match
    n_rows = min(len(df_gen_overlap), len(df_known_overlap))
    df_gen_overlap = df_gen_overlap.iloc[:n_rows]
    df_known_overlap = df_known_overlap.iloc[:n_rows]

    # Compare all values
    diffs_mask = ~np.isclose(df_gen_overlap.values,
                             df_known_overlap.values,
                             atol=atol, rtol=rtol, equal_nan=True)

    n_total = diffs_mask.size
    n_diffs = np.count_nonzero(diffs_mask)
    pct_match = 100 * (1 - n_diffs / n_total)

    # Molecule-level agreement
    row_match = np.all(~diffs_mask, axis=1)
    pct_rows_match = 100 * np.mean(row_match)

    print(f"\nSummary:")
    print(f"- Total values compared: {n_total}")
    print(f"- Mismatched values: {n_diffs}")
    print(f"- % values matching: {pct_match:.2f}%")
    print(f"- % molecules with all descriptors matching: {pct_rows_match:.2f}%")

    # Column order check (binary)
    same_order = gen_cols == known_cols
    print(f"Column order identical: {same_order}")
            
# verify mordred generation algorithm on original AGILE data
compare_descriptors(
    "../data/LNPDB_for_AGILE/outputs/finetuning_set_smiles_plus_features_plus_features.csv", # features generated by code on AGILE smiles
    "../data/LNPDB_for_AGILE/AGILE/data/data/finetuning_set_smiles_plus_features.csv" # features provided with AGILE data
)

Total known features: 813
Total generated features: 813
Overlap: 813

Summary:
- Total values compared: 975600
- Mismatched values: 0
- % values matching: 100.00%
- % molecules with all descriptors matching: 100.00%
Column order identical: True


# 5. Evaluating models on AGILE and LNPDB data

Once the molecular feature descriptors are generated, AGILE splits can make predictions on delivery efficacy for LNPDB data.

The following blocks move`infer_vis_LNPDB.py` into AGILE from `LNPDB/data/LNPDB_for_AGILE/scripts`, evaluate the fine-tuned AGILE models on the test cross-evaluation splits and make predictions on data.

## 5.1 Preparing infer_vis_LNPDB.py

In [7]:
# Copy infer_vis_LNPDB.py and infer yaml into AGILE model folders
scripts_dir = Path("../data/LNPDB_for_AGILE/scripts")
infer_py = scripts_dir / "infer_vis_LNPDB.py"
infer_yaml_template = scripts_dir / "infer_vis/config_finetune.yaml"
print(infer_py)

# Ensure files exist
assert infer_py.exists(), "infer_vis_LNPDB.py not found in scripts/"
assert infer_yaml_template.exists(), "infer_vis/config_finetune.yaml not found in scripts/"

# Copy infer_vis_LNPDB.py into AGILE
shutil.copy(infer_py, agile_dir / "infer_vis_LNPDB.py")

logging.info("infer_vis_LNPDB.py copied.")

INFO:root:infer_vis_LNPDB.py copied.


../data/LNPDB_for_AGILE/scripts/infer_vis_LNPDB.py


## 5.2 Evaluate fine-tuned models on test splits

In [8]:
# do cross-validation on the test splits
def test_split_validation() -> None:
    """
    Run the AGILE cross-evaluation splits on respective test splits
    """
    result_loss = []
    result_rmse = []
    result_corr = []
    cv_number_list = [0, 1, 2, 3, 4]
    # run infer_vis
    for cv_number in cv_number_list:
        model_path = Path(f"agile_lnp_hela_cv_{cv_number}")
        yaml_path = f"../data/LNPDB_for_AGILE/AGILE/finetune/agile_lnp_hela_cv_{cv_number}/checkpoints/config_finetune.yaml"
        # replace yaml value based on dataset_name
        with open(yaml_path, "r") as f:
                yaml_config = yaml.safe_load(f)
        # Modify the parameter
        yaml_config['task_name'] = f"df{cv_number}_test"
        # Write the updated data back to the file
        with open(yaml_path, 'w') as file:
            yaml.safe_dump(yaml_config, file, default_flow_style=False, sort_keys=False)
        command = ["python", "infer_vis_LNPDB.py", str(model_path)]
        result = subprocess.run(command, cwd=agile_dir, capture_output=True, text=True)
        try:
            import re
            stdout_vals = [float(x) for x in re.findall(r"[+-]?(?:\d+\.\d*|\.\d+|\d+)(?:[eE][+-]?\d+)?", result.stdout)]

        except ValueError:
            print(f"Warning: could not convert stdout/stderr to float for {dataset_name} CV {cv_number}")
            continue

        # uncomment to show the direct AGILE output and errors
        # print(result.stdout)
        # print(result.stderr)

        result_loss.append(stdout_vals[-17])
        result_rmse.append(stdout_vals[-16])
        result_corr.append(stdout_vals[-15])

    print("\n=== Averaged results across CV splits ===")
    loss_avg = sum(result_loss) / len(result_loss) if result_loss else float("nan")
    rmse_avg = sum(result_rmse) / len(result_rmse) if result_rmse else float("nan")
    corr_avg = sum(result_corr) / len(result_corr) if result_corr else float("nan")
    print(f"Average loss = {loss_avg:.4f}, Average RMSE = {rmse_avg:.4f}, Pearson Corr = {corr_avg:.4f}")

test_split_validation()


=== Averaged results across CV splits ===
Average loss = 10.1025, Average RMSE = 3.1737, Pearson Corr = 0.3261


## 5.3 Make predictions on LNPDB data

In [6]:
# evaluate cross-validation splits on dataset
def cross_evaluate() -> None:
    """
    Run the AGILE cross-evaluation splits on LNPDB data
    """
    cv_number_list = [0, 1, 2, 3, 4]
    # datasets to evaluate
    datasets = ["BL_2023_heldout_data", "LM_2019_heldout_data", "SL_2020_heldout_data", "ZC_2023_heldout_data"]
    dataset_metrics = {dataset_name: {"loss": [], "rmse": [], "corr": []} for dataset_name in datasets}
    for dataset_name in datasets:
        # run infer_vis
        for cv_number in cv_number_list:
            model_path = Path(f"agile_lnp_hela_cv_{cv_number}")
            yaml_path = Path(f"../data/LNPDB_for_AGILE/AGILE/finetune/agile_lnp_hela_cv_{cv_number}/checkpoints/config_finetune.yaml")
            # replace yaml value based on dataset_name
            with open(yaml_path, "r") as f:
                yaml_config = yaml.safe_load(f)
            # Modify the parameter
            yaml_config['task_name'] = dataset_name
            # Write the updated data back to the file
            with open(yaml_path, 'w') as file:
                yaml.safe_dump(yaml_config, file, default_flow_style=False, sort_keys=False)
            command = ["python", "infer_vis_LNPDB.py", str(model_path)]
            result = subprocess.run(command, cwd=agile_dir, capture_output=True, text=True)
            try:
                import re
                stderr_vals = [float(x) for x in re.findall(r"[+-]?(?:\d+\.\d*|\.\d+|\d+)(?:[eE][+-]?\d+)?", result.stdout)]
            except ValueError:
                print(f"Warning: could not convert stdout/stderr to float for {dataset_name} CV {cv_number}")
                continue

            # uncomment to show the direct AGILE output and errors
            # print(result.stdout)
            # print(result.stderr)
            
            dataset_metrics[dataset_name]["loss"].append(stderr_vals[-3])
            dataset_metrics[dataset_name]["rmse"].append(stderr_vals[-2])
            dataset_metrics[dataset_name]["corr"].append(stderr_vals[-1])

    print("\n=== Averaged results across CV splits ===")
    for dataset_name, metrics in dataset_metrics.items():
        loss_avg = sum(metrics["loss"]) / len(metrics["loss"]) if metrics["loss"] else float("nan")
        rmse_avg = sum(metrics["rmse"]) / len(metrics["rmse"]) if metrics["rmse"] else float("nan")
        corr_avg = sum(metrics["corr"]) / len(metrics["corr"]) if metrics["corr"] else float("nan")
        print(f"{dataset_name}: Average loss = {loss_avg:.4f}, Average RMSE = {rmse_avg:.4f}, Pearson Corr = {corr_avg:.4f}")

cross_evaluate()

# model prediction and ranking is saved inside the model folder in ../AGILE/finetune


=== Averaged results across CV splits ===
BL_2023_heldout_data: Average loss = 32.5733, Average RMSE = 5.6852, Pearson Corr = -0.0215
LM_2019_heldout_data: Average loss = 47.7940, Average RMSE = 6.9071, Pearson Corr = -0.0158
SL_2020_heldout_data: Average loss = 47.9252, Average RMSE = 6.8049, Pearson Corr = -0.2242
ZC_2023_heldout_data: Average loss = 47.1363, Average RMSE = 6.7901, Pearson Corr = 0.2167
